## Update of current stock prices

In [1]:

import yfinance as yf
from datetime import datetime
import pandas
# TODO add specialisation for folder from where to take data

tickers = pandas.read_csv('./data/securities.csv')

def update_ticker_price():
    for index, row in tickers.iterrows():
        ticker = yf.Ticker(row['ticker']).info
        if ticker['quoteType'] == 'EQUITY':
            market_price = ticker['regularMarketOpen']
            currency = ticker['currency']
            current_time = datetime.now().isoformat()
            tickers.at[index,'price'] = market_price
            tickers.at[index,'price_date'] = current_time
            tickers.at[index,'price_currency'] = currency
    tickers.to_csv('./data/securities.csv',index=False)
    print('Updated ticker prices')
def get_ticker_price(ticker):
    # todo add date
    # TODO will the cvs be updated here??
    t = tickers.loc[(tickers['ticker']==ticker),'price']
    return t.iloc[0]
    
update_ticker_price()
r = get_ticker_price('ADSK')


Updated ticker prices


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

endpoint = 'https://api.apilayer.com/exchangerates_data/latest'
access_key = os.getenv('APILAYER_ACCESS_TOKEN')

currencies = {}

def get_rates_for_date(base_currency,target_currency):
    # TODO add date to the function
    if currency in currencies:
        response= currencies[currency]
    else:
        params = {
            'base': DEFAULT_CURRENCY,
            'symbols': currency
        }
        headers= {
            "apikey": access_key
        }
        response = requests.get(endpoint, params=params,headers=headers).json()
        print(response)
        currencies[currency] = response
    return currencies[currency] 

In [3]:
import requests
import numpy
DEFAULT_CURRENCY='EUR'


import pandas
orders = pandas.read_csv('./data/orders.csv')
total_profit=0
for index, row in orders.iterrows():
    currency = orders.at[index,'currency']
    
    if currency != DEFAULT_CURRENCY:
        response = get_rates_for_date(DEFAULT_CURRENCY,currency)
        rate = response['rates'][currency]
        # TODO convert at the date of order
    else:
        rate = 1
    orders.at[index,'buy_total_to_currency'] = orders.at[index,'price_total'] / rate + orders.at[index,'tax_for_transaction'] / rate
    today_price = get_ticker_price(orders.at[index,'ticker'])/ rate
    orders.at[index,'today_evaluation'] = today_price * orders.at[index,'amount'] 
    orders.at[index,'profit'] = orders.at[index,'today_evaluation']  + orders.at[index,'buy_total_to_currency']
    orders.at[index,'percent_of_profit'] = orders.at[index,'profit'] / orders.at[index,'buy_total_to_currency'] *100
    if (numpy.isnan(orders.at[index,'profit']) == False): total_profit += orders.at[index,'profit']
print(total_profit)
orders.groupby(['ticker']).sum()
# <!-- calculate totals for each ticker -->
# <!-- update data with current prices -->
# <!-- convert data to default_currency (euro) -->
# <!-- calculate total profit -->
# <!-- save data to the table -->


{'success': True, 'timestamp': 1684649403, 'base': 'EUR', 'date': '2023-05-21', 'rates': {'USD': 1.082363}}
{'success': True, 'timestamp': 1684649403, 'base': 'EUR', 'date': '2023-05-21', 'rates': {'RUB': 86.589442}}
-534.827508048049


,date,amount,price_total,tax_for_transaction,currency,buy_total_to_currency,today_evaluation,profit,percent_of_profit
ticker,,,,,,,,,
ADSK,2020-12-18T16:28,3,-697.9400,2.80,USDRUB,-551.919730,379.308964,-172.610766,-74.500905
EPAM,2021-12-03T10:10,1,-622.0000,1.87,USD,-572.940871,221.496855,-351.444016,61.340364
FXIT,2021-12-03T10:01,3,-35991.9999,107.97,RUB,-414.415766,0.000000,0.000000,0.000000
PTC,2021-02-17T11:06,1,-146.1900,0.44,USD,-134.659075,123.886349,-10.772726,8.000000


In [4]:
import pandas as pd

# Create a sample DataFrame
df = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age': [25, 30, 35],
    'Salary': [50000, 60000, 70000]
})

# Add a row at the bottom with the sum of each column
df.loc['Total'] = df.sum(axis=0)

# Print the result
print(df)

                  Name  Age  Salary
0                Alice   25   50000
1                  Bob   30   60000
2              Charlie   35   70000
Total  AliceBobCharlie   90  180000
